<div style='text-align: center; font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;'>
  <span style='font-size: 40px;'>🚀</span>
  <span style='font-weight: normal; font-size: 32px; color: #3AB77A;'>Desafio Creditas -</span>
  <span style='font-weight: bold; font-size: 32px; color: #3AB77A;'> Cientista de Dados Pleno</span>
</div>
<div style='text-align: center; font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;'>
  <span style='font-weight: normal; font-size: 19px; color: #F;'>Etapa 4: Construção de features</span>
</div>

<br><br>

<div style='color: #3AB77A; text-align: center; font-size: 28px; font-weight: bold'>
  <p>
    Conteúdo desse Notebook
  </p>
</div>
<div style='text-align: center; font-size: 20px;'>
  <span style='font-size: 20px;'>✅</span>
  <span>Feature Engineering</p>
</div>
<div style='text-align: center; font-size: 20px;'>
  <span style='font-size: 20px;'>✅</span>
  <span>Apresentação da lógica de construção das features</p>
</div>
<div style='text-align: center; font-size: 20px;'>
  <span style='font-size: 20px;'>✅</span>
  <span>Normalização dos dados via arcsinh</p>
</div>

In [ ]:
import pandas as pd
import numpy as np
import sys, os
sys.path.insert(0, os.path.abspath(".."))

from utils.plot_utils import plotter as ptz

pd.set_option('display.max_columns', 500)
from utils.eda_utils import (
    get_all_information,
    StartEda
)

from utils.feature_utils import (
    ModelUtils,
    create_pipeline,
    get_class
)

import warnings
warnings.filterwarnings("ignore")

# Dessa vez utilizando a base bronze
df = pd.read_csv('../data/bronze_dataset.csv')

get_all_information(df=df, column_filter='pre_approved', column_target='sent_to_analysis')

df = df[df['pre_approved'] == 1].drop(['pre_approved'],axis=1)

# Realizando os processos de input de NA baseado na regra identificada anteriormente
control = StartEda(df=df, direct_input_cols=['collateral_debt','informed_restriction'])
df = control.df

# Analisando a problematica dos dados #
De todos os casos nessa base, reprovados temos 19820
De todos os casos nessa base, pré-aprovado temos 14976
Que realmente aprovamos para a analise de crédito foram 3262
Que realmente reprovamos para a analise de crédito foram 11714


In [2]:
columns_ignore= [
 'id',
 'dishonored_checks',
 'expired_debts',
 'banking_debts',
 'commercial_debts',
 'protests',
 'informed_restriction',
 'form_completed',
 'verified_restriction'
]

columns_without = [
 'dishonored_checks',
 'expired_debts',
 'banking_debts',
 'commercial_debts',
 'protests',
 'informed_restriction',
 'form_completed',
 'verified_restriction',
 'id'
]

object_preprocessing = ModelUtils(
    df=df, 
    target='sent_to_analysis',
    columns_ignore=columns_ignore, 
    columns_without=columns_without
)
X_train, y_train, X_test, y_test, X_val, y_val = object_preprocessing.train_test_val();

pipe_prep = create_pipeline(
    df=X_train,
    columns_ignore=columns_ignore,
    columns_include_without_transformation=columns_without,
    ordinal_order=None,
    numerical_scaler=None,
    is_eda=True
) ; pipe_prep[0]

X_train_eda = pipe_prep[0].fit_transform(X_train)
X_train_eda = pd.DataFrame(X_train_eda, 
                           columns=pipe_prep[1].tolist() + pipe_prep[2].tolist() + pipe_prep[3].tolist() + pipe_prep[4].tolist())

X_train_eda['set'] = 'train'

X_val_eda = pipe_prep[0].transform(X_val)
X_val_eda = pd.DataFrame(X_val_eda, 
                         columns=pipe_prep[1].tolist() + pipe_prep[2].tolist() + pipe_prep[3].tolist() + pipe_prep[4].tolist())

X_val_eda['set'] = 'val'

X_eda = pd.concat([X_train_eda, X_val_eda],axis=0)

for col in X_eda.columns:
    if col in ['monthly_income', 'collateral_value', 'loan_amount', 'collateral_debt', 'monthly_payment']:
        X_eda[col] = X_eda[col].astype('float64')
    elif col in ['age','auto_year']:
        X_eda[col] = X_eda[col].astype('int64')

X_eda = X_eda.reset_index(drop=True)
y_combined = pd.concat([y_train, y_val]).reset_index(drop=True)
X_eda['target'] = y_combined        

Ignorando essas colunas tanto para OneHot quanto para Numerical: ['id', 'dishonored_checks', 'expired_debts', 'banking_debts', 'commercial_debts', 'protests', 'informed_restriction', 'form_completed', 'verified_restriction']
['age', 'monthly_income', 'collateral_value', 'loan_amount', 'collateral_debt', 'monthly_payment', 'auto_year']
['city', 'state', 'informed_purpose', 'auto_brand', 'auto_model', 'channel', 'zip_code', 'landing_page_product', 'gender', 'education_level']
DataFrames criados sendo numericas:7, one_hot:10, ordinal:0
Pipelines criados, criando of FeatureUnion
[FeatureUnion] ...... (step 1 of 3) Processing pipe_num, total=   0.0s
[FeatureUnion] ...... (step 2 of 3) Processing pipe_hot, total=   0.0s
[FeatureUnion]  (step 3 of 3) Processing pipe_sem_mexer, total=   0.0s


<br><br>

<div style='text-align: center; font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;'>
  <span style='font-size: 30px;'>📊</span>
  <span style='font-weight: bold; font-size: 32px; color: #3AB77A;'>Feature Engineering: Variáveis financeiras</span>
</div>

<br>

<div style='text-align: center; font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;'>
  <span style='font-weight: bold; font-size: 25px; color: #8ADAB4;'>Contexto</span>
</div>

<p style='text-align: center; font-family: "Helvetica Neue", Helvetica, Arial, sans-serif; font-style: italic; color: #8ADAB4; font-size: 20px;'>
  Construindo features financeiras
</p>

In [3]:
loan_percentile_75 = np.percentile(X_train['loan_amount'], 75)
collateral_debt_percentile_50 = np.percentile(X_train['collateral_debt'], 50)
collateral_value_percentile_50 = np.percentile(X_train['collateral_value'], 50)
collateral_value_debt_percentile_50 = np.percentile(X_train['collateral_value'] + X_train['collateral_debt'], 50)

In [13]:
print(
    loan_percentile_75,
    collateral_debt_percentile_50,
    collateral_value_percentile_50,
    collateral_value_debt_percentile_50
)

18715.05 0.0 23000.0 24649.0


In [4]:
# QtdSalariosMinimos para diminuir
X_eda['QtdSalariosMinimos'] = X_eda.apply(lambda x: x['monthly_income']/1512, axis=1)
X_eda['ValorGarantiaRealAuto'] = X_eda.apply(lambda x: x['collateral_value'] - x['collateral_debt'],axis=1)
X_eda['PropLoanGarantiaVeiculo'] = X_eda.apply(lambda x: x['loan_amount']/x['collateral_value'] if x['collateral_value'] > 0 else -1,axis=1)
X_eda['PropLoanReal'] = X_eda.apply(lambda x: x['loan_amount']/x['ValorGarantiaRealAuto'] if x['ValorGarantiaRealAuto'] > 0 else -1,axis=1)
X_eda['PropLoanMonthlyIncome'] = X_eda.apply(lambda x: x['loan_amount']/x['monthly_income'] if x['monthly_income'] > 0 else -1,axis=1)

# Monthly_payment
X_eda['PropVlrParcelaMensal'] = X_eda.apply(lambda x: x['monthly_payment']/x['monthly_income'] if x['monthly_income'] > 0 else -1,axis=1)
X_eda['PropMonthyPaymentLoan'] = X_eda.apply(lambda x: x['monthly_payment']/x['loan_amount'] if x['loan_amount'] > 0 else -1,axis=1)
X_eda['PropLoanCollateral'] = X_eda.apply(lambda x: 
                                          ((x['loan_amount'] + x['collateral_debt'])/(x['monthly_income']*12)) 
                                          if x['monthly_income'] > 0 else -1,axis=1)

X_eda['PropDebtIncome'] = X_eda.apply(lambda x: x['collateral_debt']/x['monthly_income'] if x['monthly_income'] > 0 else -1,axis=1)
X_eda['PropDebtValue'] = X_eda.apply(lambda x: x['collateral_debt']/x['collateral_value'] if x['collateral_value'] > 0 else -1,axis=1)
X_eda['PropValueIncome'] = X_eda.apply(lambda x: x['collateral_value']/x['monthly_income'] if x['monthly_income'] > 0 else -1,axis=1)
X_eda['isLoanAbove75Percent'] = X_eda.apply(lambda x: x['loan_amount']/loan_percentile_75, axis=1)
X_eda['isValueAbove50Percent'] = X_eda.apply(lambda x: x['collateral_value']/collateral_value_percentile_50, axis=1)
X_eda['isRealAbove50Percent'] = X_eda.apply(lambda x: (x['collateral_value']-x['collateral_debt'])/collateral_value_debt_percentile_50, axis=1)
X_eda['IdadeCarro'] = X_eda.apply(lambda x: 2025 - x['auto_year'], axis=1)

<br><br>

<div style='text-align: center; font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;'>
  <span style='font-size: 30px;'>📊</span>
  <span style='font-weight: bold; font-size: 32px; color: #3AB77A;'>Feature Engineering: Variáveis categorias</span>
</div>

<br>

<div style='text-align: center; font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;'>
  <span style='font-weight: bold; font-size: 25px; color: #8ADAB4;'>Contexto</span>
</div>

<p style='text-align: center; font-family: "Helvetica Neue", Helvetica, Arial, sans-serif; font-style: italic; color: #8ADAB4; font-size: 20px;'>
  Construindo features categoricas
</p>

In [5]:
df_get_info = X_train
df_get_info['target'] = y_train
df_get_info['IdadeCarro'] = df_get_info.apply(lambda x: 2025 - x['auto_year'], axis=1)
df_get_info['ValorGarantiaRealAuto'] = df_get_info.apply(lambda x: x['collateral_value'] - x['collateral_debt'],axis=1)
df_get_info['PropLoanReal'] = df_get_info.apply(lambda x: x['loan_amount']/x['ValorGarantiaRealAuto'] if x['ValorGarantiaRealAuto'] > 0 else -1,axis=1)

df_get_info_apply = df_get_info.query("target == 1").groupby('auto_brand').agg(
    mean_prop_loan_real=('PropLoanReal', 'mean'),
    mean_idade_carro=('IdadeCarro', 'mean'),
    mean_garantia_real=('ValorGarantiaRealAuto','mean')
).reset_index()

mean_prop_loan_real = dict(df_get_info_apply[['auto_brand','mean_prop_loan_real']].values)
mean_idade_carro = dict(df_get_info_apply[['auto_brand','mean_idade_carro']].values)
mean_garantia_real = dict(df_get_info_apply[['auto_brand','mean_garantia_real']].values)

In [7]:
print(
    mean_prop_loan_real,
)

{'Audi': 0.5435041518531529, 'BMW': 0.6749623561854194, 'CHERY': 0.6897570873602265, 'Chrysler': 0.4123482363127069, 'Citroën': 0.6756302083270017, 'Dodge': 0.26666222229629505, 'EFFA': 0.6185311945899138, 'Fiat': 0.6486340387776122, 'Ford': 0.6401095169912144, 'GM - Chevrolet': 0.6334866105581304, 'Honda': 0.6349733319293591, 'Hyundai': 0.6938826536151933, 'JAC': 0.7141756574248197, 'Jeep': 0.2624990625117186, 'Kia Motors': 0.6225978540650665, 'LIFAN': 0.8991976390297888, 'Land Rover': 0.6807848531285194, 'Mercedes-Benz': 0.701994051784246, 'Mitsubishi': 0.6432847897291576, 'Nissan': 21.170642894712497, 'Peugeot': 0.6403212622381467, 'Renault': 0.6046639938968722, 'Subaru': 0.49301543208962517, 'Suzuki': 0.1349206349206349, 'Toyota': 0.670929214247723, 'VW - VolksWagen': 0.6382164686300204, 'Volvo': 0.7203852403459767}


In [6]:
X_eda['Brand_MeanPropLoanReal_Credit'] = X_eda.apply(
    lambda x: -1 if not mean_prop_loan_real.get(x['auto_brand']) else mean_prop_loan_real.get(x['auto_brand']), axis=1
)

X_eda['Brand_MeanIdadeCarro_Credit'] = X_eda.apply(
    lambda x: -1 if not mean_idade_carro.get(x['auto_brand']) else mean_idade_carro.get(x['auto_brand']), axis=1
)

X_eda['Brand_MeanGarantiaReal_Credit'] = X_eda.apply(
    lambda x: -1 if not mean_garantia_real.get(x['auto_brand']) else mean_garantia_real.get(x['auto_brand']), axis=1
)

X_eda['MeanPropLoanReal_Brand'] = X_eda.apply(lambda x: x['PropLoanReal']/x['Brand_MeanPropLoanReal_Credit'],axis=1)
X_eda['MeanPropIdadeCarro_Brand'] = X_eda.apply(lambda x: x['IdadeCarro']/x['Brand_MeanIdadeCarro_Credit'],axis=1)
X_eda['MeanValorGarantiaRealAuto_Brand'] = X_eda.apply(lambda x: x['ValorGarantiaRealAuto']/x['Brand_MeanGarantiaReal_Credit'],axis=1)

# Classificando a marca
X_eda['ClassMarca'] = X_eda.apply(lambda x: get_class(x['auto_brand']),axis=1)

<br><br>

<div style='text-align: center; font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;'>
  <span style='font-size: 30px;'>📊</span>
  <span style='font-weight: bold; font-size: 32px; color: #3AB77A;'>Feature Engineering: Variáveis geográficas</span>
</div>

<br>

<div style='text-align: center; font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;'>
  <span style='font-weight: bold; font-size: 25px; color: #8ADAB4;'>Contexto</span>
</div>

<p style='text-align: center; font-family: "Helvetica Neue", Helvetica, Arial, sans-serif; font-style: italic; color: #8ADAB4; font-size: 20px;'>
  Construindo features relacionado a geografia do lead
</p>

In [7]:
df_get_info['PropLoanMonthlyIncome'] = df_get_info.apply(lambda x: x['loan_amount']/x['monthly_income'] if x['monthly_income'] > 0 else -1,axis=1)
df_get_info['PropLoanGarantiaVeiculo'] = df_get_info.apply(lambda x: x['loan_amount']/x['collateral_value'] if x['collateral_value'] > 0 else -1,axis=1)
df_get_info_zip_code_grouped = df_get_info.groupby('zip_code').agg(
    count_total_cases=('zip_code','count'),
    count_credit_cases=('target','sum'),
    mean_loan_income_target=('PropLoanMonthlyIncome', lambda x: x[df_get_info.loc[x.index, 'target'] == 1].mean()),
    mean_loan_collateral_target=('PropLoanGarantiaVeiculo', lambda x: x[df_get_info.loc[x.index, 'target'] == 1].mean())
).reset_index()

df_get_info_zip_code_grouped['mean_loan_income_target'] = df_get_info_zip_code_grouped['mean_loan_income_target'].fillna(-1)
df_get_info_zip_code_grouped['mean_loan_collateral_target'] = df_get_info_zip_code_grouped['mean_loan_collateral_target'].fillna(-1)
df_get_info_zip_code_grouped['PropHitRate'] = df_get_info_zip_code_grouped.apply(lambda x: x['count_credit_cases']/x['count_total_cases'],axis=1)


mean_loan_income = dict(df_get_info_zip_code_grouped[['zip_code','mean_loan_income_target']].values)
mean_loan_collateral_target = dict(df_get_info_zip_code_grouped[['zip_code','mean_loan_collateral_target']].values)
hit_rate = dict(df_get_info_zip_code_grouped[['zip_code','PropHitRate']].values)

In [8]:
X_eda['ZipCode_MeanPropLoanIncome_Credit'] = X_eda.apply(
    lambda x: -1 if not mean_loan_income.get(x['zip_code']) else mean_loan_income.get(x['zip_code']), axis=1
)

X_eda['ZipCode_MeanLoanCollateralTarget_Credit'] = X_eda.apply(
    lambda x: -1 if not mean_loan_collateral_target.get(x['zip_code']) else mean_loan_collateral_target.get(x['zip_code']), axis=1
)

X_eda['ZipCode_HitRate_Credit'] = X_eda.apply(
    lambda x: -1 if not hit_rate.get(x['zip_code']) else hit_rate.get(x['zip_code']), axis=1
)

X_eda['MeanPropLoanMonthly_ZipCode'] = X_eda.apply(lambda x: x['PropLoanMonthlyIncome']/x['ZipCode_MeanPropLoanIncome_Credit'],axis=1)
X_eda['MeanPropLoanGarantiaVeiculo_ZipCode'] = X_eda.apply(lambda x: x['PropLoanGarantiaVeiculo']/x['ZipCode_MeanLoanCollateralTarget_Credit'],axis=1)

In [9]:
df_salarios = pd.read_excel('../data/salario_medio_empresas_UF.xlsx')

X_eda.rename({
    'state':'UF'
},axis=1, inplace=True)

X_eda = X_eda.merge(
            df_salarios[['UF','salario_medio_mensal ','numero_de_empresas_atuantes']],
            on='UF',
            how='left'
)

X_eda['PropIncomeSalarioMedioMensal'] = X_eda.apply(lambda x: x['monthly_income']/x['salario_medio_mensal '],axis=1)

In [10]:
df_class_purpose = pd.read_csv('../data/class_purpose.csv').drop(['Unnamed: 0'],axis=1)

X_eda['id'] = X_eda['id'].astype('int64')
X_eda = X_eda.merge(df_class_purpose[['id', 'purpose_normalized', 'ClassPurpose']], on=['id'], how='left')
X_eda['ClassPurpose'] = X_eda.apply(lambda x: 
        'nothing' if x['purpose_normalized'] == 'nada a declarar' else x['ClassPurpose'],axis=1
)

In [11]:
for col in ['monthly_income', 'collateral_value', 'loan_amount', 'collateral_debt', 'monthly_payment']:
    X_eda[col] = np.arcsinh(X_eda[col])